In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

In [2]:
from google.colab import drive

In [3]:
# get size of the variable in megabytes
def get_variable_size(var):
    display(sys.getsizeof(var)/ (1024**2))

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
X_train = pd.read_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/X_train_4.csv')

In [7]:
X_valid = pd.read_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/X_valid_4.csv')

In [23]:
y_train_log = pd.read_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/y_train_log_4.csv').squeeze("columns")

In [24]:
y_valid_log = pd.read_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/y_valid_log_4.csv').squeeze("columns")

In [34]:
y_train_log.columns

Index(['Unnamed: 0', 'trip_duration_log'], dtype='object')

In [33]:
type(y_train_log)

pandas.core.frame.DataFrame

## 5.

### 5.1.

In [26]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train_log)

LinearRegression()

In [27]:
y_train_predict = model.predict(X_train)
display(y_train_predict)

array([[7.28531674e+05, 6.25623088e+00],
       [7.27505932e+05, 5.86520156e+00],
       [7.26408833e+05, 5.77635568e+00],
       ...,
       [7.27447310e+05, 6.15504219e+00],
       [7.30000763e+05, 5.53020005e+00],
       [7.29490889e+05, 6.19938915e+00]])

In [28]:
y_valid_predict = model.predict(X_valid)
display(y_valid_predict)

array([[7.28678266e+05, 5.99976557e+00],
       [7.27947103e+05, 5.70736869e+00],
       [7.28833040e+05, 6.19622123e+00],
       ...,
       [7.29947484e+05, 6.45579266e+00],
       [7.29781419e+05, 6.60420998e+00],
       [7.31097890e+05, 6.22865283e+00]])

In [29]:
def calculate_rmsle(y_true, y_pred):
    return round(metrics.mean_squared_error(y_true, y_pred, squared=False), 2)

In [30]:
calculate_rmsle(y_train_log, y_train_predict)

210480.22

In [31]:
calculate_rmsle(y_valid_log, y_valid_predict)

210470.3

### 5.2.

In [16]:
poly_features = preprocessing.PolynomialFeatures(degree=2, include_bias=False)

X_train_poly = poly_features.fit_transform(X_train)
X_valid_poly = poly_features.fit_transform(X_valid)

linear_reg_poly = linear_model.LinearRegression()
linear_reg_poly.fit(X_train_poly, y_train_log)

y_train_poly_predict = linear_reg_poly.predict(X_train_poly)
print(f"y_train_poly_predict: {y_train_poly_predict}")
y_valid_poly_predict = linear_reg_poly.predict(X_valid_poly)
print(f"y_valid_poly_predict: {y_valid_poly_predict}")

print("RMSLE train_poly:", calculate_rmsle(y_train_log, y_train_poly_predict))
print("RMSLE train:", calculate_rmsle(y_valid_log, y_valid_poly_predict))

y_train_poly_predict: [[7.29664188e+05 6.12466230e+00]
 [7.24034211e+05 5.80877225e+00]
 [7.13490536e+05 5.62098888e+00]
 ...
 [7.28042320e+05 5.91285271e+00]
 [7.26961981e+05 5.22358627e+00]
 [7.31913297e+05 6.27398546e+00]]
y_valid_poly_predict: [[7.28646657e+05 5.86037059e+00]
 [7.28269132e+05 5.39682444e+00]
 [7.20994794e+05 5.80252699e+00]
 ...
 [7.28830771e+05 6.47661560e+00]
 [7.28717381e+05 6.86568538e+00]
 [7.30812333e+05 6.29690244e+00]]
RMSLE train_poly: 210446.94
RMSLE train: 210529.01


### 5.3.